### Env Config

In [2]:
import os
while not os.path.exists('.gitignore'):
	os.chdir(os.path.abspath(os.path.join('../')))
from common_imports import *

In [24]:
# Aproximação da quantidade de linhas esperadas no arquivo final 
TOTAL_ROW_COUNT = 100000
# Randomizar amostras coletadas de cada arquivo
DO_RANDOMIZE_SAMPLE_ROWS = True

# Exportar os arquivos tratados ou não
DO_EXPORT_FILES = True
# Tipo de arquivo a ser gerado
OUTPUT_FILE_TYPE = 'csv' # 'csv' ou 'parquet'

# Rodar ou não a visualização dos recursos (Demora muito mais tempo)
RUN_RESOURCE_VISUALIZATION = False

In [25]:
def export_to_file(df: DataFrame, path: str):
	if DO_EXPORT_FILES:
		if not os.path.exists(path):
			os.makedirs(path)
		if OUTPUT_FILE_TYPE == 'csv':
			# df.write.csv(f'{path}/df.csv', header=True, mode='overwrite')
			df.to_csv(f'{path}/df.csv', index=False)
		else:
			# df.write.parquet(f'{path}/df.parquet', mode='overwrite')
			df.to_parquet(f'{path}/df.parquet', index=False)

### Criando dataframe

In [26]:
files = os.listdir('_data/raw/operacoes-credito')
files.sort()

sample_size = TOTAL_ROW_COUNT // len(files)

arrow_tables = []
for file in files:
	print(f'Processando o arquivo {file}')

	temp_df = pd.read_parquet(f'_data/raw/operacoes-credito/{file}', engine='pyarrow').sample(n=sample_size, random_state=random.randint(1, 1000))
	table = pa.Table.from_pandas(temp_df)
	arrow_tables.append(table)

combined_table = pa.concat_tables(arrow_tables)

df = combined_table.to_pandas()

Processando o arquivo 201206.parquet
Processando o arquivo 201207.parquet
Processando o arquivo 201208.parquet
Processando o arquivo 201209.parquet
Processando o arquivo 201210.parquet
Processando o arquivo 201211.parquet
Processando o arquivo 201212.parquet
Processando o arquivo 201301.parquet
Processando o arquivo 201302.parquet
Processando o arquivo 201303.parquet
Processando o arquivo 201304.parquet
Processando o arquivo 201305.parquet
Processando o arquivo 201306.parquet
Processando o arquivo 201307.parquet
Processando o arquivo 201308.parquet
Processando o arquivo 201309.parquet
Processando o arquivo 201310.parquet
Processando o arquivo 201311.parquet
Processando o arquivo 201312.parquet
Processando o arquivo 201401.parquet
Processando o arquivo 201402.parquet
Processando o arquivo 201403.parquet
Processando o arquivo 201404.parquet
Processando o arquivo 201405.parquet
Processando o arquivo 201406.parquet
Processando o arquivo 201407.parquet
Processando o arquivo 201408.parquet
P

In [27]:
df.count()

data_base                         99864
uf                                99864
cliente                           99864
ocupacao                          99864
cnae_secao                        99864
porte                             99864
modalidade                        99864
numero_de_operacoes               99864
carteira_ativa                    99864
carteira_inadimplida_arrastada    99864
ativo_problematico                99864
indexador_modalidade              96759
possui_modalidade                 99864
dtype: int64

### Ajuste de tipagem dos dados

In [28]:
df.dtypes

data_base                         object
uf                                object
cliente                           object
ocupacao                          object
cnae_secao                        object
porte                             object
modalidade                        object
numero_de_operacoes               object
carteira_ativa                    object
carteira_inadimplida_arrastada    object
ativo_problematico                object
indexador_modalidade              object
possui_modalidade                   bool
dtype: object

In [29]:
MONETARY_COLS = ['a_vencer_ate_90_dias', 'a_vencer_de_91_ate_360_dias', 'a_vencer_de_361_ate_1080_dias', 'a_vencer_de_1081_ate_1800_dias', 'a_vencer_de_1801_ate_5400_dias', 'a_vencer_acima_de_5400_dias', 'vencido_acima_de_15_dias', 'carteira_ativa', 'carteira_inadimplida_arrastada', 'ativo_problematico']

CATEGORY_COLS = ['uf', 'tcb', 'sr', 'ocupacao', 'cnae_secao', 'cnae_subclasse', 'porte', 'modalidade', 'origem', 'indexador']

QUANTITY_COLS = ['numero_de_operacoes']

DATE_COLS = ['data_base']

for column in df.columns:
    if column in MONETARY_COLS:
        df[column] = df[column].str.replace(',', '.').astype(float)
        df = df.rename(columns={column: f'vl_{column}'})
    elif column in CATEGORY_COLS:
        df = df.rename(columns={column: f'ct_{column}'})
    elif column in QUANTITY_COLS:
        df[column] = df[column].str.replace('<= ', '').astype(int)
        df = df.rename(columns={column: f'qt_{column}'})
    elif column in DATE_COLS:
        df[column] = pd.to_datetime(df[column], format='%Y-%m-%d')
        df = df.rename(columns={column: f'dt_{column}'})

In [30]:
df.dtypes

dt_data_base                         datetime64[ns]
ct_uf                                        object
cliente                                      object
ct_ocupacao                                  object
ct_cnae_secao                                object
ct_porte                                     object
ct_modalidade                                object
qt_numero_de_operacoes                        int64
vl_carteira_ativa                           float64
vl_carteira_inadimplida_arrastada           float64
vl_ativo_problematico                       float64
indexador_modalidade                         object
possui_modalidade                              bool
dtype: object

In [31]:
df['cliente'].value_counts()

cliente
PJ    85840
PF    14024
Name: count, dtype: int64

In [32]:
df = df.rename(columns={'cliente': 'ct_classificacao'})

#### Assim percebemos que já existe uma coluna com a classificação do cliente, então vamos retirar essa informação das outras colunas

In [15]:
if RUN_RESOURCE_VISUALIZATION:
	df.groupBy('ct_porte').count().orderBy('count', ascending=False).show()
	df.groupBy('ct_modalidade').count().orderBy('count', ascending=False).show()
	df.groupBy('ct_ocupacao').count().orderBy('count', ascending=False).show()
	df.groupBy('ct_cnae_secao').count().orderBy('count', ascending=False).show()
	df.groupBy('ct_cnae_subclasse').count().orderBy('count', ascending=False).show()

In [16]:
# df = df.withColumn('ct_porte', trim(substring(col('ct_porte'), 6, 100)))
# df = df.withColumn('ct_modalidade', trim(substring(col('ct_modalidade'), 6, 100)))
# df = df.withColumn('ct_ocupacao', trim(substring(col('ct_ocupacao'), 6, 100)))
# df = df.withColumn('ct_cnae_secao', trim(substring(col('ct_cnae_secao'), 6, 100)))
# df = df.withColumn('ct_cnae_subclasse', trim(substring(col('ct_cnae_subclasse'), 6, 100)))

### Exportando para zona Trusted

In [33]:
export_to_file(df, '_data/trusted/operacoes-credito')

In [18]:
df = df.withColumn('vl_carteira_ativa_n_arrastada', col('vl_carteira_ativa') - col('vl_carteira_inadimplida_arrastada'))

In [19]:
if RUN_RESOURCE_VISUALIZATION:
	df.select('vl_carteira_ativa_n_arrastada').describe().show()

In [20]:
if RUN_RESOURCE_VISUALIZATION:
	df.agg(
		sum(when(col('vl_a_vencer_acima_de_5400_dias') != 0.00, 1).otherwise(0)).alias('count_acima_5400'),
		sum(when(col('vl_a_vencer_de_1801_ate_5400_dias') != 0.00, 1).otherwise(0)).alias('count_1801_5400'),
		sum(when(col('vl_a_vencer_de_361_ate_1080_dias') != 0.00, 1).otherwise(0)).alias('count_361_1080'),
		sum(when(col('vl_a_vencer_de_91_ate_360_dias') != 0.00, 1).otherwise(0)).alias('count_91_360'),
		sum(when(col('vl_a_vencer_de_1081_ate_1800_dias') != 0.00, 1).otherwise(0)).alias('count_1081_1800'),
		sum(when(col('vl_a_vencer_ate_90_dias') != 0.00, 1).otherwise(0)).alias('count_ate_90')
	).show()

In [21]:
df = df.withColumn(
    'ct_faixa_meses_ate_vencimento',
    when(col('vl_a_vencer_acima_de_5400_dias') != 0.00, '> 180')
     .when(col('vl_a_vencer_de_1801_ate_5400_dias') != 0.00, '36-180')
     .when(col('vl_a_vencer_de_1081_ate_1800_dias') != 0.00, '18-36')
     .when(col('vl_a_vencer_de_361_ate_1080_dias') != 0.00, '12-18')
     .when(col('vl_a_vencer_de_91_ate_360_dias') != 0.00, '3-12')
     .when(col('vl_a_vencer_ate_90_dias') != 0.00, '0-3')
     .otherwise(None)
)

In [22]:
if RUN_RESOURCE_VISUALIZATION:
	df.groupBy('ct_faixa_meses_ate_vencimento').count().orderBy('count', ascending=False).show()

In [23]:
df = df.withColumn('vl_media_carteira_ativa_por_operacao', col('vl_carteira_ativa') / col('qt_numero_de_operacoes'))

In [24]:
if RUN_RESOURCE_VISUALIZATION:
	df.groupBy('vl_media_carteira_ativa_por_operacao').count().orderBy('count', ascending=False).show()

In [25]:
df = df.withColumn('vl_media_carteira_inadimplida_por_operacao', col('vl_carteira_inadimplida_arrastada') / col('qt_numero_de_operacoes'))

In [26]:
if RUN_RESOURCE_VISUALIZATION:
	df.groupBy('vl_media_carteira_inadimplida_por_operacao').count().orderBy('count', ascending=False).show()

In [27]:
df.dtypes

root
 |-- dt_data_base: date (nullable = true)
 |-- ct_uf: string (nullable = true)
 |-- ct_tcb: string (nullable = true)
 |-- ct_sr: string (nullable = true)
 |-- ct_classificacao: string (nullable = true)
 |-- ct_ocupacao: string (nullable = true)
 |-- ct_cnae_secao: string (nullable = true)
 |-- ct_cnae_subclasse: string (nullable = true)
 |-- ct_porte: string (nullable = true)
 |-- ct_modalidade: string (nullable = true)
 |-- ct_origem: string (nullable = true)
 |-- ct_indexador: string (nullable = true)
 |-- qt_numero_de_operacoes: integer (nullable = true)
 |-- vl_a_vencer_ate_90_dias: decimal(20,2) (nullable = true)
 |-- vl_a_vencer_de_91_ate_360_dias: decimal(20,2) (nullable = true)
 |-- vl_a_vencer_de_361_ate_1080_dias: decimal(20,2) (nullable = true)
 |-- vl_a_vencer_de_1081_ate_1800_dias: decimal(20,2) (nullable = true)
 |-- vl_a_vencer_de_1801_ate_5400_dias: decimal(20,2) (nullable = true)
 |-- vl_a_vencer_acima_de_5400_dias: decimal(20,2) (nullable = true)
 |-- vl_vencido_

### Exportando para zona Refined

In [28]:
export_to_file(df, '_data/refined/operacoes-credito')

In [29]:
spark.stop()